In [1]:
%load_ext autoreload
%autoreload 2
directoryN = "/ipynbmorris/"
import sys; 
CPBdir = "../../../../cpb3101/CPlantBox"
sys.path.append(CPBdir+"/src");
sys.path.append(CPBdir);
sys.path.append("../../..");sys.path.append(".."); 
sys.path.append(CPBdir+"/src/python_modules");
sys.path.append("../build-cmake/cpp/python_binding/") # dumux python binding
sys.path.append("../../build-cmake/cpp/python_binding/")
sys.path.append("../modules/") # python wrappers 
import numpy as np
import json
import plantbox as pb
import importlib
importlib.reload(pb)
import plantbox
importlib.reload(plantbox)
from phloem_flux import PhloemFluxPython 
import pickle
from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
from joblib import Parallel, delayed
import os 
from helpUqr import *

simDuration=18; condition="dry"
weatherX = weather(simDuration, condition, 1)
assert weatherX['Qlight']>0

#			default value of env variable, to re-set at runtime
cs = 390e-6; 

Chl = 55,; 
oi = 210e-3;#leaf internal [O2] [mol mol-1]

SPAD= 100.0
chl_ = (0.114 *(SPAD**2)+ 7.39 *SPAD+ 10.6)/10


Q10 = 2.; #double TrefQ10 = 20;//to compute effect of T on growth (see CN-wheat, residual respiration @Barillot 2016, appendix)
#psiMax = 0;  
psiMin = -2000*(1/0.9806806);#limit wat. pot. in xylem for water-limited growth, [cm]
KMfu = 0.2; #@see C_fluxes,Michaelis menten coef for active sucrose usage
Vmaxloading = 0.019872;#mmol cm-1 d-1 for leaf blade 1cm wide
CSTimin = 0.4;#minimum CST value below which there is no sink of sucrose
beta_loading = 1;#@see C_fluxes, feedback effect of C_ST on Q_FL
Mloading = 0.2;#@see C_fluxes,Michaelis menten coef for Fl
Gr_Y = 0.75;#growth efficiency
psi_osmo = -10000*1.0197
delta_osmo_min = -psi_osmo - psiMin



namesVars_phlo = ['Q10','delta_osmo_min','psiMin','KMfu','Vmaxloading','CSTimin',
              'beta_loading', 'Mloading','Gr_Y','rhoSucrose',
             'kx_st','kr_st','Across','krm2','krm1','rmax']
namesLim_phlo = [Q10,delta_osmo_min,psiMin,KMfu,Vmaxloading,
                 CSTimin,beta_loading,Mloading,Gr_Y,1,
             1,1,1,1,1,1 ]

namesVars_xyl = ['Chl','oi','fw1r','k_fw1','psi_t,crit,1',
              'kchl1', 'kchl2',
             'kg1','kjmax', 'alpha','omega',
             'gamma0', 'gamma1', 'gamma2','g0',
             'kx_x','kr_x']
namesLim_xyl = [[0.5,2],[0.5,2],[0.5,2],[0.5,2],[0.5,2],
                 [0.5,2],[0.5,2],[0.5,2],[0.5,2],[0.02/0.4,0.95/0.4],
             [0.04/0.6,0.98/0.6],[0.5,2],[0.5,2],[0.5,2],[0.5,2],
                [0.5,2] ,[0.5,2]]
problem = {
    'num_vars': len(namesVars_xyl),
    'names': namesVars_xyl,
    'bounds': namesLim_xyl
}
namesVars = namesVars_xyl + namesVars_phlo

##### TO CHANGE ######
Nreps = 11
if (simDuration > 10) and (condition == "dry"):
    Nreps = 10
reps =2**Nreps
maxcore = os.cpu_count()
print('nreps, cpu',Nreps, maxcore)
######################

param_values = saltelli.sample(problem, reps)
#print(len(param_values))

#print(param_values[13])
#assert False




Yev  = np.array([])
Yag = np.array([])


didDo = 0

Env variables at 18 d 0 hrs :
 {'TairC': 30.27, 'TairK': 303.41999999999996, 'Pair': 1070.0, 'es': 43.11959850459574, 'Qlight': 0.00096, 'rbl': 40.0, 'rcanopy': 3.333333333333333, 'rair': 18.36187383298185, 'ea': 18.97262334202213, 'cs': 0.00035, 'RH': 0.44, 'p_mean': -422.4238769868247, 'vg': [0.08, 0.43, 0.04, 1.6, 50]}
nreps, cpu 10 32


/tmp/ipykernel_302191/666253406.py:89: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, reps)


In [2]:

def SobolTest2(myid,weatherX, varNames, varLims, simDuration,condition, testType):
    
    assert testType=="Xylem" or testType == "Phloem"
    test_values = varLims
    DictVal = {}
    for key in varNames:
        for value in test_values:
            DictVal[key] = value
            test_values.remove(value)
            break
            
    pl = pb.MappedPlant(seednum = 2) 
    path = CPBdir+"/modelparameter/plant/"
    name = "Triticum_aestivum_test_2021"

    pl.readParameters(path + name + ".xml")
    depth = 60
    sdf = pb.SDF_PlantBox(np.Inf, np.Inf, depth )
    pl.setGeometry(sdf) # creates soil space to stop roots from growing out of the soil
    pl.initialize(verbose = True)#, stochastic = False)
    pl.simulate(simDuration, False)#, "outputpm15.txt")

    picker = lambda x,y,z : max(int(np.floor(-z)),-1)   
    pl.setSoilGrid(picker)  # maps segment


    r = PhloemFluxPython(pl,psiXylInit =-1000,ciInit = 0.5 *weatherX['cs'])


    hp = max([tempnode[2] for tempnode in r.get_nodes()]) /100
    weatherX = weather(simDuration, condition, hp)

    plant =setDefaultVals(r,weatherX,DictVal)
    p_bot = weatherX["p_mean"] + depth/2
    p_top = weatherX["p_mean"] - depth/2
    xXx = np.linspace(p_top, p_bot, depth)
    
    plant.Q10 = DictVal['Q10']# X[0]
    plant.psi_osmo_proto= -DictVal['delta_osmo_min'] - DictVal['psiMin']
    plant.psiMin = DictVal['psiMin']
    plant.KMfu = DictVal['KMfu']
    plant.Vmaxloading = DictVal['Vmaxloading']
    plant.CSTimin = DictVal['CSTimin']
    plant.beta_loading = DictVal['beta_loading']
    plant.Mloading = DictVal['Mloading']
    plant.Gr_Y= DictVal['Gr_Y']# X[8]

    plant.Patm = weatherX["Pair"]
    ##resistances
    plant.g_bl = resistance2conductance(weatherX["rbl"],weatherX,r) / r.a2_bl
    plant.g_canopy = resistance2conductance(weatherX["rcanopy"],weatherX,r) / r.a2_canopy
    plant.g_air = resistance2conductance(weatherX["rair"],weatherX,r) / r.a2_air
    plant.gm = 0.01
    
    plant.usePg4Fw = True
    #r.doWiggle= True;r.DoSun2012=False
    plant.alternativeAn = True
    plant.Theta_ag = 0.9
    plant.limMaxErr = 1/100
    plant.maxLoop = 10000
    plant.xXx = xXx
    return plant

In [3]:
i=191
plant = SobolTest2(i,weatherX, 
     namesVars, #all the var names
     list(param_values[didDo + i])+list(namesLim_phlo), #all the var values
     simDuration, condition,"Xylem")

MappedPlant::initialize 
Seed::initialize: Plant 
Seed::initialize: Basal root type #4 was not defined, using tap root parameters instead
Env variables at 18 d 0 hrs :
 {'TairC': 30.27, 'TairK': 303.41999999999996, 'Pair': 1070.0, 'es': 43.11959850459574, 'Qlight': 0.00096, 'rbl': 40.0, 'rcanopy': 1.0003084710599022, 'rair': 47.33294129610276, 'ea': 18.97262334202213, 'cs': 0.00035, 'RH': 0.44, 'p_mean': -422.4238769868247, 'vg': [0.08, 0.43, 0.04, 1.6, 50]}
Tiller stem type #4 was not defined, using main stem parameters instead, Exchange zone in roots: kr > 0 until 0.8cm from root tip
Kr is constant per subtype of organ type, for root, subtype 0 = 5.4885e-05 1 day-1 
Kx is constant per subtype of organ type, for root, subtype 0 = 0.140775 cm3 day-1 
Exchange zone in roots: kr_st > 0 until 0.8cm from root tip 1 1
Exchange zone in roots: kr > 0 until 0.8cm from root tip


In [4]:

plant.solve_photosynthesis(sim_time_ = simDuration, sxx_= plant.xXx, 
                                   cells_ = True,ea_ = weatherX["ea"],es_ = weatherX["es"],
                verbose_ = False, doLog_ = False,TairC_= weatherX["TairC"] ,outputDir_= "./results"+directoryN)


In [63]:
plant.seg_leaves_idx = plant.get_segments_index(4)
plant.Chl_ = plant.Chl[0]

Eac = 59430;  Eaj = 37000;  Eao = 36000
Eard = 53000; Eav = 58520;
Edj =  220000; Edv = 220000;
Kc_ref = 302e-6; Ko_ref = 256e-3;
Rd_ref = 0.32e-6;Tref = 293.2;
S = 700;
Mh2o = 18;
R_ph = 83.143
rho_h2o = 1000;
M_Chla = 893.51

In [64]:
Vcmax = np.full(len(plant.seg_leaves_idx),np.nan)
for li_ in range(1):#len(plant.seg_leaves_idx)):
    Vcrefmax = (plant.VcmaxrefChl1* plant.Chl_ + plant.VcmaxrefChl2)*1e-6 ;

    #Vcmax
    expo1 = np.exp(Eav /(R_ph*0.1*Tref)*(1. - Tref/plant.TleafK));
    expo2 =np.exp((S* plant.TleafK - Edv)/(R_ph*0.1* plant.TleafK));
    Vcmax[li_] = Vcrefmax * expo1 / (1. + expo2); #Eq 11
    #for Vc
    #mmol mmol-1 * exp(mJ mmol-1/(hPa cm3K−1mmol−1 *(mJ/(hPa/cm3))*K)*(-))=mmol mmol-1 * exp(-)
    Ko = Ko_ref *  np.exp(Eao/(R_ph*0.1*Tref)*(1.-Tref/plant.TleafK)); #Eq 9
    Kc = Kc_ref * np.exp(Eac/(R_ph*0.1*Tref)*(1.-Tref/plant.TleafK)) ;#Eq 9
    delta = plant.gamma0* (1.+ plant.gamma1*(plant.TleafK - Tref) + plant.gamma2*((plant.TleafK - Tref)**2.) ) ;#Eq 10

    #electron transport rate
    #Jrefmax
    Jrefmax = Vcrefmax * plant.a3 ;#q 25
    #Jmax
    expo1 = np.exp(Eaj /(R_ph*0.1*Tref)*(1. - Tref/plant.TleafK));
    expo2 = np.exp((S * plant.TleafK - Edj)/(R_ph *0.1*plant.TleafK));
    Jmax = min(Jrefmax * expo1 / (1. + expo2), Jrefmax); #Eq 24
    coefa = plant.theta;
    coefb = -(plant.alpha * plant.Qlight + Jmax);
    coefc = plant.alpha * plant.Qlight * Jmax;
    dis = coefb*coefb - (4.*coefa*coefc);
    if (dis < 0):
        print("Photosynthesis::initVcVjRd : root for J not found");

    J =  ((-coefb- (dis**0.5))/(2.*coefa));#rostamza2020, Bonan2019Chap11
    if (J < 0):
        print("Photosynthesis::loopCalcs : J < 0");
    print(J)
  

Photosynthesis::initVcVjRd : root for J not found


TypeError: '<' not supported between instances of 'complex' and 'int'

In [66]:
li_=1
#Vcrefmax = (plant.VcmaxrefChl1* plant.Chl_ + plant.VcmaxrefChl2)*1e-6
print( plant.theta,plant.alpha,coefb,dis, coefa,coefc,plant.alpha , plant.Qlight , Jmax,Jrefmax,plant.TleafK)

1.167919921875 0.52783203125 -0.0008811995140194172 -1.2484787020950308e-07 1.167919921875 1.929414073763079e-07 0.52783203125 0.0009 0.00040615068589441725 0.00040615068589441725 303.41999999999996


In [11]:
weatherX["TairC"]

30.27

In [12]:
weatherX["TairK"]

303.41999999999996

In [18]:
plant.psi_air #cm

896840.1294216727

In [ ]:
#theta:1.167919921875,alpha: 0.52783203125